# Cell Type Marker Genes DotPlots
Code written and curated by Raph Gibbs, Megan Duffy, Monica Mesecar, and Dom Acri

In [ ]:
! date

## Install and Import Packages:

In [ ]:
#Install Pandas
!pip install pandas

#Install ScanPy
!pip install scanpy

In [ ]:
#Import ScanPy
import scanpy as sc

#Import Warnings/Filter 
import warnings
warnings.simplefilter('ignore')

sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()

In [ ]:
#Import Annotation data bject
import anndata as AnnData

In [ ]:
#Import NumPy
import numpy as np

#Import where module
from numpy import where

In [ ]:
#Import relevant Pandas functions
from pandas import read_csv, concat

In [ ]:
#Import MatPlotLib/PyPlot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context

In [ ]:
#Import Seaborn
import seaborn as sns

In [ ]:
#Set directories and variables
cohort= 'aging'
home_dir = '/labshare/raph/datasets/adrd_neuro'
wrk_dir = f'{home_dir}/{cohort}'
results_dir = f'{wrk_dir}/demux'

# in files 
in_file  = '/data/ADRD/brain_aging/phase1/demux/aging.pegasus.leiden_085.subclustered.h5ad'
markers_json='data/ADRD/brain_aging/exploration/human_brain_cell_markers_LNG.json'

# constants
leiden_resolution = 0.85

DEBUG= False

## Create Shorthand for Files/File Paths

## Load Data

In [ ]:
%%time
adata = sc.read(in_file)
print(adata)
if DEBUG:
    display(adata.obs.head())

new_cluster_names = ['OD-0', 'ExN-1','Astro-2','Micro-3', 'OPC-4','OD-ATAC-5', 'OD-6', 'ExN-7', 'ExN-8', 'ExN-9', 'InN-10', 
    'ExN-11', 'InN-12', 'ExN-13', 'Astro-14', 'ExN-15', 'InN-16', 'InN-17','InN-18', 'VLMC-19', 'Uncertain-20', 
    'ExN-21', 'ExN-22', 'InN-23', 'ExN-24', 'OD-25', 'InN-26', 'InN-27', 'ExN-28', 'PeriVasc-29', 'ExN-30',
    'ExN-31', 'ExN-32','Radial Glia-33','Uncertain-ATAC-34','OD-ATAC-35', 'OPC-36']

adata.rename_categories('leiden_MultiVI', new_cluster_names)

In [ ]:
#Remove misannotated nuclei
mask = ((adata.obs['broad_celltype'] == 'SPN') & (adata.obs['Brain_region'] != 'Putamen')) 

In [ ]:
display(mask==True)

In [ ]:
mask = ((adata.obs['broad_celltype'] == 'SPN') & (adata.obs['Brain_region'] != 'Putamen')) | (adata.obs['broad_celltype'] == 'Other')
adata = adata[~mask].copy()

In [ ]:
subset = adata[adata.obs['broad_celltype'] == 'SPN']
unique_values = subset.obs['Brain_region'].unique()
print(unique_values)

## Create UMAP 

In [ ]:
#Plot UMAP by modality

#Figure creation, size, and resolution:
with rc_context({'figure.figsize': (8, 8), 'figure.dpi': 100}): 
    #Specify color palette:
    plt.style.use('seaborn-v0_8-bright')
    #Specify data object source and what variable to color by:
    sc.pl.umap(adata, color=['leiden_labels_085'], 
               #Specify y/n frame and legend location:
               frameon=False, legend_loc='on data', save='P1_Leiden_UMAP.png')

In [ ]:
#Create list of marker genes 
CNS_marker_genes=['AQP4','GFAP', 'FOXJ1','CLU','SOX9', 'GJA1', 'FLT1', 'HES1', 'HES5', 'CLDN5','RBFOX3', 'SYP','NEFH','DLG4','SLC17A7','GRIN1','FOXP2','CLSTN2','FEZF2','LAMP5','RORB','THEMIS','CUX2','RGS6','SLC6A1','GAD1','ADARB2','LHX6','VIP','PVALB','SST','P2RY12','CSF1R','OLIG1','PDGFRA','MBP','PLP1','OPALIN','VIM','PPP1R1B','DRD1','DRD2','ADORA2A']

In [ ]:
#Marker dotplot by UMAP cluster number 
#Create Figure. Set size and resolution.
with plt.rc_context({'figure.figsize': (12, 12), 'figure.dpi': 300}):
    plt.style.use('seaborn-v0_8-talk')  # Use this available style
    sc.pl.dotplot(
        adata, CNS_marker_genes, groupby='leiden_labels_085',
        color_map='Blues', swap_axes=False, use_raw=False,
        standard_scale='var', save='P1_Leiden_DotPlot.pdf'
    )


In [ ]:
#Marker dotplot by broad cell type annotation 
with plt.rc_context({'figure.figsize': (12, 12), 'figure.dpi': 300}):
    plt.style.use('seaborn-v0_8-talk')  # Use this available style
    sc.pl.dotplot(
        adata, CNS_marker_genes, groupby='broad_celltype',
        color_map='Blues', swap_axes=False, use_raw=False,
        standard_scale='var', save='P1_Broad_DotPlot.pdf'

    )